In [68]:
import json
import pandas as pd
from pprint import pprint


In [ ]:
## Preprosamiento previo. De un gran dataset escogemos 5 subreddits para hacer el analisis grupal e individual
## League of legends

## Analizamos la data. Limpiamos y preprocesamos

In [81]:
## Leemos la data

file_name = 'sample_data.json'

entry_df = pd.read_json(file_name,lines = True)
entry_df.head()

,author,author_cakeday,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,...,id,is_submitter,link_id,parent_id,permalink,retrieved_on,score,stickied,subreddit,subreddit_id
0,Dethcola,NaN,,Clairemont,A quarry,True,0,1506816000,None,0,...,dnqik14,False,t3_73ieyz,t3_73ieyz,/r/sandiego/comments/73ieyz/best_place_for_gra...,1509189606,3,False,sandiego,t5_2qq2q
1,PennyBotV2,NaN,flair2-penny1,The Bot,[Salutations! I'm not sure what you said.](htt...,True,0,1506816000,None,0,...,dnqik15,False,t3_73g740,t1_dnqiiv7,/r/RWBY/comments/73g740/ruby_gives_greetings_a...,1509189606,3,False,RWBY,t5_2vhg0
2,Sir_Firebum,NaN,HOU-wagon,Astros Bandwagon,I got into baseball at about he same time Matt...,True,0,1506816000,None,0,...,dnqik16,False,t3_73hlwn,t1_dnqc3lu,/r/baseball/comments/73hlwn/matt_cain_in_his_f...,1509189606,2,False,baseball,t5_2qm7u
3,deanzynut,NaN,woodcutting,99 wcing btw,FUCKING TORY,True,0,1506816000,None,0,...,dnqik17,True,t3_73gw9b,t1_dnqdo99,/r/2007scape/comments/73gw9b/carls_platebody/d...,1509189606,18,False,2007scape,t5_2wbww
4,OfullOstomacheO,NaN,None,None,I see a water dragon,True,0,1506816000,None,0,...,dnqik18,False,t3_73i6z3,t3_73i6z3,/r/mildlyinteresting/comments/73i6z3/this_bric...,1509189606,1,False,mildlyinteresting,t5_2ti4h


In [74]:
## Podemos borrar algunas columnas no relevantes para el analisis

entry_df = entry_df.drop(['author','controversiality','author_flair_css_class','author_flair_text','can_gild','created_utc','id','permalink','retrieved_on','stickied'],axis = 1)
entry_df.head()

,author_cakeday,body,distinguished,edited,gilded,is_submitter,link_id,parent_id,score,subreddit,subreddit_id
0,NaN,A quarry,None,0,0,False,t3_73ieyz,t3_73ieyz,3,sandiego,t5_2qq2q
1,NaN,[Salutations! I'm not sure what you said.](htt...,None,0,0,False,t3_73g740,t1_dnqiiv7,3,RWBY,t5_2vhg0
2,NaN,I got into baseball at about he same time Matt...,None,0,0,False,t3_73hlwn,t1_dnqc3lu,2,baseball,t5_2qm7u
3,NaN,FUCKING TORY,None,0,0,True,t3_73gw9b,t1_dnqdo99,18,2007scape,t5_2wbww
4,NaN,I see a water dragon,None,0,0,False,t3_73i6z3,t3_73i6z3,1,mildlyinteresting,t5_2ti4h


In [99]:
## Borramos las filas que tengan como comentario borrado [Deleted]
df = entry_df.query('body != "[deleted]" and body != "[removed]"')

df.head()

,author,author_cakeday,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,...,id,is_submitter,link_id,parent_id,permalink,retrieved_on,score,stickied,subreddit,subreddit_id
0,Dethcola,NaN,,Clairemont,A quarry,True,0,1506816000,None,0,...,dnqik14,False,t3_73ieyz,t3_73ieyz,/r/sandiego/comments/73ieyz/best_place_for_gra...,1509189606,3,False,sandiego,t5_2qq2q
1,PennyBotV2,NaN,flair2-penny1,The Bot,[Salutations! I'm not sure what you said.](htt...,True,0,1506816000,None,0,...,dnqik15,False,t3_73g740,t1_dnqiiv7,/r/RWBY/comments/73g740/ruby_gives_greetings_a...,1509189606,3,False,RWBY,t5_2vhg0
2,Sir_Firebum,NaN,HOU-wagon,Astros Bandwagon,I got into baseball at about he same time Matt...,True,0,1506816000,None,0,...,dnqik16,False,t3_73hlwn,t1_dnqc3lu,/r/baseball/comments/73hlwn/matt_cain_in_his_f...,1509189606,2,False,baseball,t5_2qm7u
3,deanzynut,NaN,woodcutting,99 wcing btw,FUCKING TORY,True,0,1506816000,None,0,...,dnqik17,True,t3_73gw9b,t1_dnqdo99,/r/2007scape/comments/73gw9b/carls_platebody/d...,1509189606,18,False,2007scape,t5_2wbww
4,OfullOstomacheO,NaN,None,None,I see a water dragon,True,0,1506816000,None,0,...,dnqik18,False,t3_73i6z3,t3_73i6z3,/r/mildlyinteresting/comments/73i6z3/this_bric...,1509189606,1,False,mildlyinteresting,t5_2ti4h


In [85]:
## Buscamos relacion entre comentarios y respuesta entre el dataset
## Vamos a tratar de usar PRAW y redditAPI para obtener estas



In [100]:
import re

## Vamos a elimitar el formato de hipervinculo en un comentario y agregar una []()

def processHyperLink(comment):        
    print(comment)
    p = re.compile(r"(.*)[.*]", re.DOTALL)        
    m = p.search(comment)    
    if m:
        name = m.group(1)
        print(name)
    else:
        print(comment)
    
        

df['body'] = df['body'].apply(processHyperLink)

A quarry
A quarry
[Salutations! I'm not sure what you said.](http://imgur.com/9TtaInH) 
 

[Salutations! I'm not sure what you said.](http://imgur
I got into baseball at about he same time Matt Cain started playing in the majors. Crazy to see him go. I teared up a bit too.
I got into baseball at about he same time Matt Cain started playing in the majors. Crazy to see him go. I teared up a bit too
FUCKING TORY
FUCKING TORY
I see a water dragon 
I see a water dragon 
Wait. The Michigan what? Where is this? Is this like U of M club or a just state of Michigan?
Wait
ye fam
ye fam
143417804| &gt; United States Anonymous (ID: LIAKFEVH)

&gt;&gt;143412250 (OP)
oldfag here

2016: Hillary
2012: Obama
2008: Obama
2004: Kerry
2000: Buchanan
1996: Dole
1992: Bush
1988: Bush
1984: Reagan
			
143417804| &gt; United States Anonymous (ID: LIAKFEVH)

&gt;&gt;143412250 (OP)
oldfag here

2016: Hillary
2012: Obama
2008: Obama
2004: Kerry
2000: Buchanan
1996: Dole
1992: Bush
1988: Bush
1984: Reagan
			
Tha

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
comments = []
file_object  = open('sample_data.json') 
for line in file_object:
    comments.append(json.loads(line))

In [24]:
comments[458]

{'author': 'banterbeast',
 'author_flair_css_class': 'hibananew',
 'author_flair_text': 'mikke121 - XB1 Asia',
 'body': 'Drones should be kept the same to ensure consistency. If skins are introduced they could be detected by defenders more easily or blend into the environment more.',
 'can_gild': True,
 'controversiality': 0,
 'created_utc': 1506816014,
 'distinguished': None,
 'edited': False,
 'gilded': 0,
 'id': 'dnqike1',
 'is_submitter': False,
 'link_id': 't3_73i78p',
 'parent_id': 't3_73i78p',
 'permalink': '/r/Rainbow6/comments/73i78p/drone_skins_from_a_new_alpha_pack/dnqike1/',
 'retrieved_on': 1509189612,
 'score': 0,
 'stickied': False,
 'subreddit': 'Rainbow6',
 'subreddit_id': 't5_2t1bl'}

In [40]:
preprocComments = []
for item in comments:
    if(item['score']>=2 or item['score']<=-2):
        preprocComments.append(item)

In [41]:
len(preprocComments)

5196

In [42]:
groupedBySubR = {}
for item in preprocComments:
    if(item['subreddit'] not in groupedBySubR.keys()):
        newList = []
        newList.append(item)
        groupedBySubR[item['subreddit']] = newList
    else:
        groupedBySubR[item['subreddit']].append(item)

In [44]:
len(groupedBySubR)

1561

In [52]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...


True

In [53]:
from nltk.corpus import reuters
len(reuters.words())

1720901

In [54]:
files = [f for f in reuters.fileids() if 'training' in f]
corpus = [reuters.raw(fileids=[f]) for f in files]
len(corpus)

7769

In [55]:
corpus[0]

'BAHIA COCOA REVIEW\n  Showers continued throughout the week in\n  the Bahia cocoa zone, alleviating the drought since early\n  January and improving prospects for the coming temporao,\n  although normal humidity levels have not been restored,\n  Comissaria Smith said in its weekly review.\n      The dry period means the temporao will be late this year.\n      Arrivals for the week ended February 22 were 155,221 bags\n  of 60 kilos making a cumulative total for the season of 5.93\n  mln against 5.81 at the same stage last year. Again it seems\n  that cocoa delivered earlier on consignment was included in the\n  arrivals figures.\n      Comissaria Smith said there is still some doubt as to how\n  much old crop cocoa is still available as harvesting has\n  practically come to an end. With total Bahia crop estimates\n  around 6.4 mln bags and sales standing at almost 6.2 mln there\n  are a few hundred thousand bags still in the hands of farmers,\n  middlemen, exporters and processors.\n  

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [57]:
X = vectorizer.fit_transform(corpus)

In [58]:
X.shape

(7769, 26000)

In [59]:
type(X)

scipy.sparse.csr.csr_matrix